The goal of this notebook is to show the pipeline:
1) Retrieving match data from Riot's TFT API
2) Processing and saving the data
3) Building a TFT model for predicting fights

In [9]:
import pandas as pd
import json
import itertools
from tft_api.sample_response import sample_match_data
from tft_api.tft_api import get_match_by_match_id, get_match_ids_by_puuid, TEST_PUUID, TEST_MATCH_ID
import copy
import random

"""
If running in prod, we would be fetching data from Riots api

get_match_ids_by_puuid(TEST_PUUID)
get_match_by_match_id(TEST_MATCH_ID)

Instead we will use sample match data
"""

# Get players in a completed match and sort them by their placement
players = sample_match_data["info"]["participants"]
players = sorted(players, key=lambda i: i['placement'])
players = [{
	"placement": player['placement'],
	"units": player['units'],
} for player in players]

# print(players)

IndentationError: unindent does not match any outer indentation level (tft_api.py, line 58)

In [2]:
# REMOVE inactive traits and unused data
for player in players:
    player['units'] = [[unit['character_id'], unit['items'], unit['rarity'], unit['tier']] for unit in player['units']]
    

# Create match_pair_data:
#
# [Player that won 
#  (0 = first player, 1 = second),
#   player1,
#   player2
# ]
match_pairs = []
for i, j in itertools.product(players, players):
    if i['placement'] < j['placement']:
        if random.randint(0, 1) == 1:
            match_pairs.append([0, i, j])
        else:
            match_pairs.append([1, j, i])
#print(match_pairs)

# Generate more match pair data by pairing players against
# strictly worse versions of themselves. (one less unit)
for player in players:
    player_copy = copy.deepcopy(player)
    while len(player_copy['units']) > 0:
        stronger = copy.deepcopy(player_copy)
        player_copy['units'].pop()
        weaker = copy.deepcopy(player_copy)
        
        if random.randint(0, 1) == 1:
            match_pairs.append([0, stronger, weaker])
        else:
            match_pairs.append([1, weaker, stronger])


NameError: name 'players' is not defined

In [3]:
# Write data to csv file
import csv

headers = ['winner', 'player0_unit0', 'player0_unit0_item0', 'player0_unit0_item1', 'player0_unit0_item2', 'player0_unit0_level', 'player0_unit1', 'player0_unit1_item0', 'player0_unit1_item1', 'player0_unit1_item2', 'player0_unit1_level', 'player0_unit2', 'player0_unit2_item0', 'player0_unit2_item1', 'player0_unit2_item2', 'player0_unit2_level', 'player0_unit3', 'player0_unit3_item0', 'player0_unit3_item1', 'player0_unit3_item2', 'player0_unit3_level', 'player0_unit4', 'player0_unit4_item0', 'player0_unit4_item1', 'player0_unit4_item2', 'player0_unit4_level', 'player0_unit5', 'player0_unit5_item0', 'player0_unit5_item1', 'player0_unit5_item2', 'player0_unit5_level', 'player0_unit6', 'player0_unit6_item0', 'player0_unit6_item1', 'player0_unit6_item2', 'player0_unit6_level', 'player0_unit7', 'player0_unit7_item0', 'player0_unit7_item1', 'player0_unit7_item2', 'player0_unit7_level', 'player0_unit8', 'player0_unit8_item0', 'player0_unit8_item1', 'player0_unit8_item2', 'player0_unit8_level', 'player1_unit0', 'player1_unit0_item0', 'player1_unit0_item1', 'player1_unit0_item2', 'player1_unit0_level', 'player1_unit1', 'player1_unit1_item0', 'player1_unit1_item1', 'player1_unit1_item2', 'player1_unit1_level', 'player1_unit2', 'player1_unit2_item0', 'player1_unit2_item1', 'player1_unit2_item2', 'player1_unit2_level', 'player1_unit3', 'player1_unit3_item0', 'player1_unit3_item1', 'player1_unit3_item2', 'player1_unit3_level', 'player1_unit4', 'player1_unit4_item0', 'player1_unit4_item1', 'player1_unit4_item2', 'player1_unit4_level', 'player1_unit5', 'player1_unit5_item0', 'player1_unit5_item1', 'player1_unit5_item2', 'player1_unit5_level', 'player1_unit6', 'player1_unit6_item0', 'player1_unit6_item1', 'player1_unit6_item2', 'player1_unit6_level', 'player1_unit7', 'player1_unit7_item0', 'player1_unit7_item1', 'player1_unit7_item2', 'player1_unit7_level', 'player1_unit8', 'player1_unit8_item0', 'player1_unit8_item1', 'player1_unit8_item2', 'player1_unit8_level']
rows = []
           
for match_pair in match_pairs:
    winner = match_pair[0]
    player1_units = match_pair[1]["units"]
    player2_units = match_pair[2]["units"]
    
    row = [winner]

    for player_num, player_units in enumerate([player1_units, player2_units]):
        for i in range(9):
            if i < len(player_units):
                unit = player_units[i]
                unit_id = unit[0]
                unit_items = unit[1]
                unit_level = unit[3]
            else:
                unit_id = "None"
                unit_items = [0,0,0]
                unit_level = 0
            
            row.append(unit_id)
            items = unit_items + [0]*(3-len(unit_items))
            row = row + items
            
            row.append(unit_level)

    rows.append(row)


with open('./data/preprocessed_training_data.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(headers)
    for row in rows:
        writer.writerow(row)

csv_file.close()

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def logistic(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)
    lr=LogisticRegression(max_iter=20000)
    lr.fit(X_train,y_train)
    y_pre=lr.predict(X_test)
    print('Accuracy : ', accuracy_score(y_test,y_pre))
        

In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# Now prep file to be interpreted by model

df_train = pd.read_csv('./data/preprocessed_training_data.csv')
df_train.head()

train_data = df_train.drop(['winner'],axis=1) # data without the winner
target = df_train['winner'] # winner column in data

one = OneHotEncoder()
one.fit(train_data)
train = one.transform(train_data)

logistic(train, target)

        

Accuracy :  0.9865470852017937
